<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Topic Modelling using Vantage and LLM
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction:</b></p>

<p style='font-size:16px;font-family:Arial'>In this comprehensive user demo, we will delve into the world of topic modeling using <b>Teradata Vantage</b> and <b>AWS Bedrock - Anthropic's Claude LLM model</b>. This cutting-edge technology empowers businesses to uncover hidden insights from vast amounts of consumer complaints data, enabling them to identify trends, improve customer satisfaction, and enhance their overall brand reputation.</p> 

<p style='font-size:16px;font-family:Arial'><b>Key Features:</b></p> 

<ol style='font-size:16px;font-family:Arial'> 
    <li><b>Scalable Data Ingestion</b>: Seamlessly integrate and process large volumes of consumer complaints data from various sources, including OpenAI, into Teradata Vantage.</li> 
    <li><b>Advanced Topic Modelling</b>: Utilize state-of-the-art topic modeling algorithms to identify and categorize underlying themes and sentiments within the complaints data, providing actionable insights.</li> 
    <li><b>Real-time Analytics</b>: Leverage Teradata Vantage's real-time analytics capabilities to monitor and respond to emerging trends and issues in consumer complaints.</li> 
    <li><b>Customizable Dashboards</b>: Create tailored dashboards to visualize and track key performance indicators (KPIs) and metrics specific to your business needs.</li> 
    <li><b>Integration with AWS Bedrock - Anthropic's Claude LLM model</b>: Seamlessly integrate with AWS Bedrock - Anthropic's Claude LLM model to collect and analyze consumer complaints data from these platforms.</li> </ol> 

<p style='font-size:16px;font-family:Arial'><b>Benefits:</b></p> 

<ol style='font-size:16px;font-family:Arial'>
<li><b>Enhanced Customer Insights</b>: Gain a deeper understanding of customer concerns and preferences, enabling data-driven decision-making.</li> 
<li><b>Improved Customer Satisfaction</b>: Identify and address recurring issues, leading to increased customer satisfaction and loyalty.</li> 
<li><b>Competitive Advantage</b>: Stay ahead of the competition by proactively addressing consumer complaints and improving brand reputation.</li> 
<li><b>Cost Savings</b>: Reduce the financial burden of handling and resolving consumer complaints by identifying and addressing root causes.</li> 
<li><b>Data-Driven Decision-Making</b>: Make informed business decisions based on actionable insights derived from topic modeling and real-time analytics.</li> </ol>

<p style = 'font-size:16px;font-family:Arial'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Configuring AWS Bedrock - Anthropic's Claude LLM model</li>
    <li>Exploring the data</li>
    <li>Topic Modelling</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>1. Configuring the environment</b>
<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Downloading and installing additional software needed</b>

In [ ]:
%%capture
!pip install -r requirements.txt --upgrade --quiet

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.2 Import the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import json, warnings
import getpass

# Visualization
import plotly.express as px

# Progress bar
from tqdm import tqdm

# Machine learning and other utilities from Teradata
from teradataml import *
from teradatagenai import TeradataAI, TextAnalyticsAI, VSManager, VectorStore, VSApi

# Requests
import requests

# Display settings
display.max_rows = 5
pd.set_option('display.max_colwidth', None)

# Set display options for dataframes, plots, and warnings
%matplotlib inline
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Connect to VantageCloud</b>
<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
print("Checking if this environment is ready to connect to VantageCloud...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env")
    # Create the Context
    eng = create_context(host=env_vars.get("host"), username=env_vars.get("username"), password=env_vars.get("my_variable"))
    execute_sql('''SET query_band='DEMO=Topic_Modelling.ipynb;' UPDATE FOR SESSION;''')
    print("Connected to VantageCloud with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud.")
    print("Please contact the support team.")

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>2.  Set up the LLM connection</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>teradatagenai</b> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local GPU compute. In this case we will use anthropoc claude-instant-v1 for low-cost, high-throughput tasks.</p>

<ol style = 'font-size:16px;font-family:Arial'>
<li><b>aws_access_key</b>: Enter your AWS access key</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

In [ ]:
access_key = getpass.getpass('aws_access_key: ')
secret_key = getpass.getpass('aws_secret_access_key: ')
region_name = getpass.getpass('region name: ')

<hr style='height:2px;border:none'>
<p style='font-size:20px;font-family:Arial'><b>3. Use the <code>TextAnalyticsAI</code> API to Perform Various Text Analytics Tasks</b></p>
<p style='font-size:16px;font-family:Arial'>You can execute the help function at the bottom of this notebook to read more about this API.</p>

In [ ]:
# Provide model details
model_name="us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Select in-database or external model
llm = TeradataAI(api_type = 'AWS',
         model_name = model_name,
         region = region_name,
         # authorization = 'Repositories.BedrockAuth'
         access_key = access_key,
         secret_key = secret_key)

obj = TextAnalyticsAI(llm=llm)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Exploring the data</b>

In [ ]:
df = DataFrame('"DEMO_ComplaintAnalysis"."Consumer_Complaints"')
df

<p style = 'font-size:16px;font-family:Arial'>Here we subset the data to get only the complaints related to Mortgage. We further analyze the issues of those complaints and pick the top 5 topics.</p>

In [ ]:
df = df[df.product == 'Mortgage']

In [ ]:
df.select(['issue', 'sub_issue', 'complaint_id']).groupby(['issue', 'sub_issue']).agg(['count']).sort('count_complaint_id', ascending = False)

<p style = 'font-size:16px;font-family:Arial'>According to the result above, we can classify the issues into the following topics:</p>

<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Mortgage Application</b>: Applying or refinancing</li>
    <li><b>Payment Trouble</b>: Issues during payment</li>
    <li><b>Mortgage Closing</b>: Finalizing the mortgage</li>
    <li><b>Report Inaccuracy</b>: Incorrect information</li>
    <li><b>Payment Struggle</b>: Difficulty paying</li>
<ul>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Topic Modelling</b>

<p style = 'font-size:16px;font-family:Arial'>Topic modeling using Large Language Models (LLMs) revolutionizes the way we understand and categorize vast collections of text data. LLMs excel in understanding the semantics and context of words, enabling sophisticated topic modeling techniques.</p>

<p style = 'font-size:16px;font-family:Arial'>Traditionally, topic modeling algorithms like Latent Dirichlet Allocation (LDA) rely on statistical patterns within documents to identify topics. However, LLMs offer a more nuanced approach. By leveraging their deep understanding of language, LLMs can extract complex themes and topics from unstructured text data with higher accuracy and flexibility.</p>

<p style = 'font-size:16px;font-family:Arial'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Reasoning with a Chain of Thought</b>: Imagine you're trying to solve a problem. With a large language model, you start with an initial idea or question. Then, you use the model's capabilities to explore related concepts, gradually connecting them together. Each step builds upon the previous one, leading you closer to understanding or solving the problem. It's like putting together puzzle pieces, one by one, until you see the whole picture.</p>

In [ ]:
tdf_topics = obj.classify(column = 'consumer_complaint_narrative', 
                          data = df,
                          labels = ['Mortgage Application',
                                    'Payment Trouble',
                                    'Mortgage Closing',
                                    'Report Inaccuracy',
                                    'Payment Struggle'])[['complaint_id','Labels','consumer_complaint_narrative']]

In [ ]:
tdf_topics

<p style = 'font-size:16px;font-family:Arial'>Now the results can be saved back to Vantage.</p>

In [ ]:
copy_to_sql(df = tdf_topics, table_name = 'topic_prediction', if_exists = 'replace')

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>5.1 Number of Complaints by Predicted Topic</b></p>

<p style = 'font-size:16px;font-family:Arial'>A graph illustrating the Number of Complaints by Predicted Topic reveals that the majority of complaints are centered around Mortgage Application, while the fewest are related to Mortgage Closing.</p>

In [ ]:
grp_gen = DataFrame('topic_prediction').select(['Labels','complaint_id']).groupby(['Labels']).agg(['count']).to_pandas()

grp_gen = grp_gen.sort_values('count_complaint_id', ascending = False)[:10]

fig = px.bar(grp_gen, x='Labels', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'Labels': 'Labels'},
             title='Number of Complaints by Predicted Topic')

# Add hover information
fig.update_traces(hovertemplate='Issue: %{x}<br>Number of Complaints: %{y:,}')

fig.show()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>6. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['topic_prediction']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024, 2025, 2026. All Rights Reserved
        </div>
    </div>
</footer>